# 00 ? Shared Environment Setup

Run this notebook once at the start of each Colab session. It installs the lightweight thesis environment, finds the repo root, and verifies CUDA access plus dataset downloads.


## Install dependencies (Colab)
Skip if you already created the `peft-thesis` conda env locally.


In [ ]:
!pip install -q -U transformers datasets evaluate accelerate peft matplotlib pandas scikit-learn


## Locate repository and seed everything


In [ ]:
import os
import sys
from pathlib import Path

def find_project_root() -> Path:
    current = Path.cwd().resolve()
    for path in [current, *current.parents]:
        if (path / "src").exists() and (path / "notebooks").exists():
            return path
    raise RuntimeError("Unable to locate the repository root. Please run this notebook from inside the project.")

PROJECT_ROOT = find_project_root()
os.chdir(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))
print(f"Project root: {PROJECT_ROOT}")


In [ ]:

from src.utils import GLOBAL_CONFIG, configure_tf32, get_device, set_seed_everywhere

set_seed_everywhere(GLOBAL_CONFIG.seed)
configure_tf32(GLOBAL_CONFIG.tf32)
device = get_device()
print(f"Seed: {GLOBAL_CONFIG.seed}")
print(f"Device: {device}")
if device.type == "cuda":
    import torch

    print(torch.cuda.get_device_name(0))


## Pre-download SST-2


In [ ]:

from src import data

sst2 = data.load_sst2()
print(sst2)
print({split: len(ds) for split, ds in sst2.items()})
